# 특강 01: Spotify API & ReccoBeats API를 활용한 음원 분석

## 학습 목표
- Spotify Client Credentials Flow를 구현해보자
- 아티스트/앨범/트랙 검색 기능을 구현해보자
- ReccoBeats API를 통해 음원 특성(Audio Features)을 분석해보자
- 음악 데이터 수집 및 분석 방법을 익혀보자

## 배경: 왜 ReccoBeats API를 사용하는가?

> **중요 변경사항**: Spotify는 2024년 11월 27일부터 Web API의 일부 엔드포인트에 대한 접근을 제한했습니다. 특히 **Audio Features API**와 **Audio Analysis API**가 영향을 받았습니다.
> 
> 이러한 변경으로 인해:
> - `GET /audio-features/{id}` - 트랙의 음향 특성 (에너지, 댄서빌리티, 템포 등)
> - `GET /audio-analysis/{id}` - 상세한 오디오 분석 데이터
> 
> 위 엔드포인트들은 더 이상 일반 개발자들이 사용할 수 없게 되었습니다.
>
> **해결책**: ReccoBeats API는 Spotify의 음원 특성 분석 기능을 대체할 수 있는 서드파티 서비스입니다. Spotify API로 트랙 정보를 검색한 후, ReccoBeats API를 통해 해당 트랙의 음향 특성을 분석할 수 있습니다.

In [90]:
# 라이브러리 import
import requests
import base64
import time
import os
import pandas as pd
from typing import Dict, List, Optional

## 1. Spotify 앱 등록

1. [Spotify for Developers](https://developer.spotify.com/) 접속
2. "Create App" 클릭  
3. **Redirect URI**: `http://localhost:8888/callback` (필수 입력, 실제로는 사용하지 않음)
4. Client ID와 Client Secret 획득

> **참고**: Client Credentials Flow에서는 Redirect URI를 실제로 사용하지 않지만, Spotify 앱 생성 시 필수 항목입니다.

## 2. API 자격 증명 설정 (.env 파일)

.env 파일을 사용하여 안전하게 자격 증명을 관리합니다.

In [91]:
def load_spotify_credentials():
    """.env 파일에서 Spotify API 자격 증명을 로드합니다."""
    env_file = '.env'
    
    # .env 파일이 없으면 템플릿 생성
    if not os.path.exists(env_file):
        template_content = """# Spotify API 자격 증명
SPOTIFY_CLIENT_ID=your_client_id_here
SPOTIFY_CLIENT_SECRET=your_client_secret_here
"""
        with open(env_file, 'w') as f:
            f.write(template_content)
        
        print("📝 .env 파일이 생성되었습니다!")
        print("   실제 Client ID와 Secret을 입력하고 다시 실행하세요")
        return None, None
    
    # .env 파일에서 읽기
    client_id = None
    client_secret = None
    
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                if key == 'SPOTIFY_CLIENT_ID':
                    client_id = value
                elif key == 'SPOTIFY_CLIENT_SECRET':
                    client_secret = value
    
    if client_id == 'your_client_id_here' or client_secret == 'your_client_secret_here':
        print("⚠️ .env 파일에 실제 값을 입력해주세요!")
        return None, None
    
    if client_id and client_secret:
        print("✅ .env 파일에서 자격 증명을 로드했습니다.")
        return client_id, client_secret
    else:
        print("❌ .env 파일에 자격 증명이 없습니다.")
        return None, None

# 자격 증명 로드
CLIENT_ID, CLIENT_SECRET = load_spotify_credentials()

✅ .env 파일에서 자격 증명을 로드했습니다.


## 3. Spotify Client Credentials Flow 구현

OAuth 서버 없이 간단하게 토큰을 발급받습니다.

In [92]:
class SpotifyClientCredentials:
    def __init__(self, client_id: str, client_secret: str):
        self.client_id = client_id
        self.client_secret = client_secret
        self.access_token = None
        self.token_expires_at = 0
        
    def get_access_token(self) -> Optional[str]:
        """Client Credentials Flow로 액세스 토큰 발급"""
        # 토큰이 유효하면 재사용
        if self.access_token and time.time() < self.token_expires_at:
            return self.access_token
            
        # 새 토큰 발급
        credentials = f"{self.client_id}:{self.client_secret}"
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        
        headers = {
            'Authorization': f'Basic {encoded_credentials}',
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        
        data = {'grant_type': 'client_credentials'}
        
        response = requests.post('https://accounts.spotify.com/api/token', 
                               headers=headers, data=data)
        
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
            expires_in = token_data.get('expires_in', 3600)
            self.token_expires_at = time.time() + expires_in - 60  # 1분 여유
            
            print("✅ 액세스 토큰을 성공적으로 발급받았습니다!")
            return self.access_token
        else:
            print(f"❌ 토큰 발급 실패: {response.status_code}")
            return None

print("✅ SpotifyClientCredentials 클래스 정의 완료!")

✅ SpotifyClientCredentials 클래스 정의 완료!


## 4. Spotify API 클라이언트 구현

Client Credentials로 발급받은 토큰으로 Spotify API를 호출하는 클래스입니다.

In [93]:
class SpotifyAPI:
    def __init__(self, client_credentials: SpotifyClientCredentials):
        self.auth = client_credentials
        self.base_url = "https://api.spotify.com/v1"
    
    def _get_headers(self) -> Optional[Dict]:
        """API 요청 헤더 생성"""
        token = self.auth.get_access_token()
        if not token:
            return None
        return {"Authorization": f"Bearer {token}"}
    
    def search(self, query: str, search_type: str = "track", limit: int = 20) -> Optional[Dict]:
        """음악 검색 (track, artist, album)"""
        headers = self._get_headers()
        if not headers:
            return None
            
        params = {
            "q": query,
            "type": search_type,
            "limit": limit
        }
        
        response = requests.get(f"{self.base_url}/search", 
                               headers=headers, params=params)
        return response.json() if response.status_code == 200 else None
    
    def get_track(self, track_id: str) -> Optional[Dict]:
        """특정 트랙 정보 가져오기"""
        headers = self._get_headers()
        if not headers:
            return None
            
        response = requests.get(f"{self.base_url}/tracks/{track_id}", 
                               headers=headers)
        return response.json() if response.status_code == 200 else None
    
    def get_artist(self, artist_id: str) -> Optional[Dict]:
        """특정 아티스트 정보 가져오기"""
        headers = self._get_headers()
        if not headers:
            return None
            
        response = requests.get(f"{self.base_url}/artists/{artist_id}", 
                               headers=headers)
        return response.json() if response.status_code == 200 else None
    
    def get_artist_top_tracks(self, artist_id: str, market: str = "US") -> Optional[Dict]:
        """아티스트의 인기곡 가져오기"""
        headers = self._get_headers()
        if not headers:
            return None
            
        params = {"market": market}
        response = requests.get(f"{self.base_url}/artists/{artist_id}/top-tracks", 
                               headers=headers, params=params)
        return response.json() if response.status_code == 200 else None

print("✅ SpotifyAPI 클래스 정의 완료!")

✅ SpotifyAPI 클래스 정의 완료!


## 5. ReccoBeats API 클라이언트 구현

ReccoBeats API를 통해 음원의 상세한 특성(Audio Features)을 분석합니다.

**ReccoBeats API 사용 플로우**:
1. Spotify 트랙 ID → ReccoBeats 트랙 검색 API로 ReccoBeats 내부 content ID 조회
2. ReccoBeats 내부 content ID → 음원 특성 분석 API로 상세 분석 결과 조회

예시: `https://api.reccobeats.com/v1/track/2670c328-c40f-45f4-80df-f48b29296deb/audio-features`

In [94]:
class ReccoBeatsAPI:
    """ReccoBeats API 클라이언트 - 음원 특성 분석용
    
    중요: ReccoBeats API는 2단계 프로세스를 사용합니다
    1. Spotify ID로 ReccoBeats 내부 content ID를 먼저 검색
    2. ReccoBeats content ID로 실제 음원 특성을 조회
    """
    
    def __init__(self):
        self.base_url = "https://api.reccobeats.com/v1"
    
    def get_content_id_by_spotify_id(self, spotify_track_id: str) -> Optional[str]:
        """1단계: Spotify 트랙 ID로 ReccoBeats content ID 조회"""
        try:
            url = f"{self.base_url}/track"
            params = {"ids": spotify_track_id}
            print(f"📡 ReccoBeats content ID 조회: {url}?ids={spotify_track_id}")
            
            response = requests.get(url, params=params, timeout=30)
          
            if response.status_code == 200:
                try:
                    data = response.json()
                    
                    # API 응답 구조 확인: {'content': [{'id': '...', ...}]}
                    if isinstance(data, dict) and 'content' in data and len(data['content']) > 0:
                        track_data = data['content'][0]  # 첫 번째 트랙 데이터
                        content_id = track_data.get('id')
                        if content_id:
                            print(f"✅ ReccoBeats content ID: {content_id}")
                            return content_id
                    
                    print(f"⚠️ 응답 구조를 인식할 수 없습니다.")
                    print(f"응답 내용: {data}")
                    return None
                    
                except ValueError as e:
                    print(f"❌ JSON 파싱 실패: {e}")
                    print(f"원본 응답: {response.text[:200]}...")
                    return None
            else:
                print(f"❌ ReccoBeats API 오류: {response.status_code}")
                if response.text:
                    print(f"오류 내용: {response.text[:200]}...")
                return None
                
        except requests.exceptions.Timeout:
            print("❌ ReccoBeats API 요청 시간 초과 (30초)")
            return None
        except requests.exceptions.ConnectionError:
            print("❌ ReccoBeats API 연결 오류 - 인터넷 연결을 확인해주세요")
            return None
        except Exception as e:
            print(f"❌ ReccoBeats API 호출 실패: {e}")
            return None
    
    def get_audio_features(self, content_id: str) -> Optional[Dict]:
        """2단계: ReccoBeats content ID로 음원 특성(Audio Features) 조회"""
        try:
            url = f"{self.base_url}/track/{content_id}/audio-features"
            print(f"📡 음원 특성 API 호출: {url}")
            
            response = requests.get(url, timeout=30)
            
            if response.status_code == 200:
                try:
                    data = response.json()
                    print("✅ 음원 특성 조회 성공!")
                    return data
                except ValueError as e:
                    print(f"❌ 음원 특성 JSON 파싱 실패: {e}")
                    return None
            else:
                print(f"❌ 음원 특성 조회 실패: {response.status_code}")
                if response.text:
                    print(f"오류 내용: {response.text[:200]}...")
                return None
                
        except requests.exceptions.Timeout:
            print("❌ 음원 특성 API 요청 시간 초과 (30초)")
            return None
        except requests.exceptions.ConnectionError:
            print("❌ 음원 특성 API 연결 오류")
            return None
        except Exception as e:
            print(f"❌ 음원 특성 API 호출 실패: {e}")
            return None
    
    def get_audio_features_by_spotify_id(self, spotify_track_id: str) -> Optional[Dict]:
        """편의 메서드: Spotify ID → ReccoBeats content ID → 음원 특성 (2단계를 한 번에)"""
        print(f"🔍 Spotify 트랙 ID로 음원 특성 분석 시작: {spotify_track_id}")
        
        content_id = self.get_content_id_by_spotify_id(spotify_track_id)
        
        if content_id:
            return self.get_audio_features(content_id)
        else:
            print("❌ ReccoBeats content ID를 찾을 수 없어 음원 특성 분석을 중단합니다.")
            return None

print("✅ ReccoBeatsAPI 클래스 정의 완료!")

✅ ReccoBeatsAPI 클래스 정의 완료!


## 6. API 클라이언트 생성 및 초기화

In [95]:
# 자격 증명이 로드되었다면 API 클라이언트 생성
if CLIENT_ID and CLIENT_SECRET:
    # Spotify 인증 객체 생성
    auth = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
    
    # API 클라이언트들 생성
    spotify = SpotifyAPI(auth)
    reccobeats = ReccoBeatsAPI()
    
    print("✅ Spotify API 클라이언트가 생성되었습니다.")
else:
    print("❌ 먼저 .env 파일에 Spotify 자격 증명을 입력하세요!")

✅ Spotify API 클라이언트가 생성되었습니다.


## 7. Spotify API 사용 예시

### 7.1 트랙 검색

In [96]:
# 트랙 검색
if CLIENT_ID and CLIENT_SECRET:
    search_results = spotify.search("NewJeans Get Up", "track", limit=5)

    if search_results and 'tracks' in search_results:
        print("=== '뉴진스 Get Up' 검색 결과 ===")
        for i, track in enumerate(search_results['tracks']['items'], 1):
            artists = ", ".join([artist['name'] for artist in track['artists']])
            print(f"{i}. {track['name']} - {artists}")
            print(f"   앨범: {track['album']['name']}")
            print(f"   발매일: {track['album']['release_date']}")
            print(f"   인기도: {track['popularity']}/100")
            print(f"   ID: {track['id']}")
            print()
    else:
        print("검색 결과가 없거나 API 호출에 실패했습니다.")
else:
    print("❌ 먼저 .env 파일에 Spotify 자격 증명을 입력하세요!")

✅ 액세스 토큰을 성공적으로 발급받았습니다!
=== '뉴진스 Get Up' 검색 결과 ===
1. Get Up - NewJeans
   앨범: NewJeans 2nd EP 'Get Up'
   발매일: 2023-07-21
   인기도: 68/100
   ID: 1wUnuiXMMvhudmzvcCtlZP

2. NEW DROP - Don Toliver
   앨범: Trending Edit Audios
   발매일: 2025-08-19
   인기도: 0/100
   ID: 0dNLRYTbYGPMwNIZCby7cV

3. ETA - NewJeans
   앨범: NewJeans 2nd EP 'Get Up'
   발매일: 2023-07-21
   인기도: 76/100
   ID: 56v8WEnGzLByGsDAXDiv4d

4. NEW DROP - Don Toliver
   앨범: Viral USA - Trending Now
   발매일: 2025-08-27
   인기도: 0/100
   ID: 4DAeFo5Sh4lxZhBYFW1NU7

5. How Sweet - NewJeans
   앨범: How Sweet
   발매일: 2024-05-24
   인기도: 76/100
   ID: 38tXZcL1gZRfbqfOG0VMTH



### 7.2 아티스트 정보 및 인기곡

In [99]:
# 아티스트 검색 및 정보
if CLIENT_ID and CLIENT_SECRET:
    artist_search = spotify.search("NewJeans", "artist", limit=1)

    if artist_search and 'artists' in artist_search and artist_search['artists']['items']:
        artist = artist_search['artists']['items'][0]
        artist_id = artist['id']
        
        print(f"=== {artist['name']} 아티스트 정보 ===")
        print(f"장르: {', '.join(artist['genres']) if artist['genres'] else '정보 없음'}")
        print(f"팔로워: {artist['followers']['total']:,}명")
        print(f"인기도: {artist['popularity']}/100")
        print()
        
        # 아티스트의 인기곡 가져오기
        top_tracks = spotify.get_artist_top_tracks(artist_id)
        
        if top_tracks and 'tracks' in top_tracks:
            print(f"=== {artist['name']} 인기곡 TOP 10 ===")
            for i, track in enumerate(top_tracks['tracks'], 1):
                print(f"{i:2d}. {track['name']}")
                print(f"    앨범: {track['album']['name']} ({track['album']['release_date'][:4]})")
                print(f"    인기도: {track['popularity']}/100")
                duration_sec = track['duration_ms'] // 1000
                print(f"    길이: {duration_sec // 60}:{duration_sec % 60:02d}")
                print()
    else:
        print("아티스트를 찾을 수 없습니다.")
else:
    print("❌ 먼저 .env 파일에 Spotify 자격 증명을 입력하세요!")

=== NewJeans 아티스트 정보 ===
장르: k-pop
팔로워: 11,250,522명
인기도: 80/100

=== NewJeans 인기곡 TOP 10 ===
 1. New Jeans
    앨범: NewJeans 'Super Shy' (2023)
    인기도: 78/100
    길이: 1:48

 2. Super Shy
    앨범: NewJeans 'Super Shy' (2023)
    인기도: 78/100
    길이: 2:34

 3. Ditto
    앨범: Ditto (2022)
    인기도: 78/100
    길이: 3:05

 4. OMG
    앨범: NewJeans 'OMG' (2023)
    인기도: 78/100
    길이: 3:32

 5. Hype Boy
    앨범: NewJeans 1st EP 'New Jeans' (2022)
    인기도: 76/100
    길이: 2:59

 6. ETA
    앨범: NewJeans 2nd EP 'Get Up' (2023)
    인기도: 76/100
    길이: 2:31

 7. How Sweet
    앨범: How Sweet (2024)
    인기도: 76/100
    길이: 3:39

 8. Attention
    앨범: NewJeans 1st EP 'New Jeans' (2022)
    인기도: 75/100
    길이: 3:00

 9. Supernatural
    앨범: Supernatural (2024)
    인기도: 75/100
    길이: 3:11

10. Bubble Gum
    앨범: How Sweet (2024)
    인기도: 72/100
    길이: 3:20



## 8. 통합 사용 예시: Spotify + ReccoBeats

### 8.1 트랙 검색 및 음원 특성 분석

In [100]:
# Spotify에서 트랙 검색 후 ReccoBeats로 음원 특성 분석
if CLIENT_ID and CLIENT_SECRET:
    search_query = "NewJeans Ditto"
    search_results = spotify.search(search_query, "track", limit=1)

    if search_results and 'tracks' in search_results and search_results['tracks']['items']:
        track = search_results['tracks']['items'][0]
        spotify_track_id = track['id']
        
        print("=== Spotify 트랙 정보 ===")
        print(f"곡명: {track['name']}")
        print(f"아티스트: {', '.join([a['name'] for a in track['artists']])}")
        print(f"앨범: {track['album']['name']}")
        print(f"발매일: {track['album']['release_date']}")
        print(f"인기도: {track['popularity']}/100")
        print(f"Spotify ID: {spotify_track_id}")
        print()
        
        # ReccoBeats에서 음원 특성 가져오기
        print("=== ReccoBeats 음원 특성 분석 ===")
        audio_features = reccobeats.get_audio_features_by_spotify_id(spotify_track_id)
        
        if audio_features:
            print(f"에너지 (Energy): {audio_features.get('energy', 'N/A'):.3f}")
            print(f"댄서빌리티 (Danceability): {audio_features.get('danceability', 'N/A'):.3f}")
            print(f"발란스 (Valence): {audio_features.get('valence', 'N/A'):.3f}")
            print(f"어쿠스틱 (Acousticness): {audio_features.get('acousticness', 'N/A'):.3f}")
            print(f"인스트루멘털 (Instrumentalness): {audio_features.get('instrumentalness', 'N/A'):.3f}")
            print(f"스피치니스 (Speechiness): {audio_features.get('speechiness', 'N/A'):.3f}")
            print(f"라이브니스 (Liveness): {audio_features.get('liveness', 'N/A'):.3f}")
            print(f"템포 (Tempo): {audio_features.get('tempo', 'N/A')} BPM")
            print(f"음조 (Key): {audio_features.get('key', 'N/A')}")
            print(f"모드 (Mode): {'Major' if audio_features.get('mode') == 1 else 'Minor' if audio_features.get('mode') == 0 else 'N/A'}")
            print(f"라우드니스 (Loudness): {audio_features.get('loudness', 'N/A')} dB")
            print(f"박자 (Time Signature): {audio_features.get('time_signature', 'N/A')}/4")
        else:
            print("⚠️ ReccoBeats에서 음원 특성을 가져올 수 없습니다.")
            print("   (트랙이 ReccoBeats 데이터베이스에 없을 수 있습니다)")
    else:
        print(f"'{search_query}' 검색 결과가 없습니다.")
else:
    print("❌ 먼저 .env 파일에 Spotify 자격 증명을 입력하세요!")

=== Spotify 트랙 정보 ===
곡명: Ditto
아티스트: NewJeans
앨범: Ditto
발매일: 2022-12-19
인기도: 78/100
Spotify ID: 3r8RuvgbX9s7ammBn07D3W

=== ReccoBeats 음원 특성 분석 ===
🔍 Spotify 트랙 ID로 음원 특성 분석 시작: 3r8RuvgbX9s7ammBn07D3W
📡 ReccoBeats content ID 조회: https://api.reccobeats.com/v1/track?ids=3r8RuvgbX9s7ammBn07D3W
✅ ReccoBeats content ID: c3eacc38-1417-421f-8978-3c56a31f2d29
📡 음원 특성 API 호출: https://api.reccobeats.com/v1/track/c3eacc38-1417-421f-8978-3c56a31f2d29/audio-features
✅ 음원 특성 조회 성공!
에너지 (Energy): 0.641
댄서빌리티 (Danceability): 0.814
발란스 (Valence): 0.183
어쿠스틱 (Acousticness): 0.027
인스트루멘털 (Instrumentalness): 0.000
스피치니스 (Speechiness): 0.111
라이브니스 (Liveness): 0.099
템포 (Tempo): 133.854 BPM
음조 (Key): 6
모드 (Mode): Minor
라우드니스 (Loudness): -5.957 dB
박자 (Time Signature): N/A/4


### 8.2 여러 트랙의 음원 특성 비교

In [101]:
# 여러 트랙의 음원 특성 비교 분석
if CLIENT_ID and CLIENT_SECRET:
    # 분석할 트랙 리스트
    tracks_to_analyze = [
        "NewJeans Ditto",
        "NewJeans Hype Boy",
        "NewJeans Attention",
        "NewJeans OMG"
    ]

    # 결과 저장용 리스트
    analysis_results = []

    for query in tracks_to_analyze:
        search_results = spotify.search(query, "track", limit=1)
        
        if search_results and 'tracks' in search_results and search_results['tracks']['items']:
            track = search_results['tracks']['items'][0]
            spotify_id = track['id']
            
            # ReccoBeats에서 음원 특성 가져오기
            audio_features = reccobeats.get_audio_features_by_spotify_id(spotify_id)
            
            if audio_features:
                # 결과 저장
                result = {
                    '곡명': track['name'],
                    '아티스트': ', '.join([a['name'] for a in track['artists']]),
                    '인기도': track['popularity'],
                    '에너지': audio_features.get('energy', None),
                    '댄서빌리티': audio_features.get('danceability', None),
                    '발란스': audio_features.get('valence', None),
                    '템포': audio_features.get('tempo', None),
                    '라우드니스': audio_features.get('loudness', None)
                }
                analysis_results.append(result)
                print(f"✅ {track['name']} 분석 완료")
            else:
                print(f"⚠️ {track['name']} - ReccoBeats 데이터 없음")
        else:
            print(f"❌ '{query}' 검색 실패")
        
        # API 호출 간격 조절 (Rate limiting 방지)
        time.sleep(0.5)

    # DataFrame으로 변환하여 표시
    if analysis_results:
        df = pd.DataFrame(analysis_results)
        print("\n=== 음원 특성 비교 분석 결과 ===")
        print(df.to_string(index=False))
        
        # 평균값 계산
        print("\n=== 평균 음원 특성 ===")
        numeric_cols = ['인기도', '에너지', '댄서빌리티', '발란스', '템포', '라우드니스']
        for col in numeric_cols:
            if col in df.columns and df[col].notna().any():
                print(f"{col}: {df[col].mean():.2f}")
    else:
        print("분석 결과가 없습니다.")
else:
    print("❌ 먼저 .env 파일에 Spotify 자격 증명을 입력하세요!")

🔍 Spotify 트랙 ID로 음원 특성 분석 시작: 3r8RuvgbX9s7ammBn07D3W
📡 ReccoBeats content ID 조회: https://api.reccobeats.com/v1/track?ids=3r8RuvgbX9s7ammBn07D3W
✅ ReccoBeats content ID: c3eacc38-1417-421f-8978-3c56a31f2d29
📡 음원 특성 API 호출: https://api.reccobeats.com/v1/track/c3eacc38-1417-421f-8978-3c56a31f2d29/audio-features
✅ 음원 특성 조회 성공!
✅ Ditto 분석 완료
🔍 Spotify 트랙 ID로 음원 특성 분석 시작: 0a4MMyCrzT0En247IhqZbD
📡 ReccoBeats content ID 조회: https://api.reccobeats.com/v1/track?ids=0a4MMyCrzT0En247IhqZbD
✅ ReccoBeats content ID: c3018a61-fdd0-4b0b-a8e0-024012e3e532
📡 음원 특성 API 호출: https://api.reccobeats.com/v1/track/c3018a61-fdd0-4b0b-a8e0-024012e3e532/audio-features
✅ 음원 특성 조회 성공!
✅ Hype Boy 분석 완료
🔍 Spotify 트랙 ID로 음원 특성 분석 시작: 2pIUpMhHL6L9Z5lnKxJJr9
📡 ReccoBeats content ID 조회: https://api.reccobeats.com/v1/track?ids=2pIUpMhHL6L9Z5lnKxJJr9
✅ ReccoBeats content ID: 21399568-0a58-420b-b946-44c53a2f1239
📡 음원 특성 API 호출: https://api.reccobeats.com/v1/track/21399568-0a58-420b-b946-44c53a2f1239/audio-features
✅ 음원 특성 조

## 9. 주의사항 및 베스트 프랙티스

### 9.1 보안 관련
- **절대로 Client Secret을 공개하지 마세요**
- `.env` 파일을 사용하여 자격 증명을 관리하세요
- `.env` 파일이 `.gitignore`에 포함되어 있는지 확인하세요

### 9.2 API 사용량 관리
- Spotify API는 Rate Limiting이 있음 (일반적으로 초당 100회 요청)
- ReccoBeats API도 자체 Rate Limiting 정책이 있을 수 있음
- 요청 간 적절한 지연 시간 추가 (`time.sleep(0.1)`)
- 대량 데이터는 배치로 처리

### 9.3 현재 사용 가능한 API 기능

#### Spotify API (2024년 11월 27일 이후)
- ✅ 트랙/아티스트/앨범 검색
- ✅ 트랙 기본 정보 (이름, 아티스트, 앨범, 발매일, 인기도)
- ✅ 아티스트 정보 (장르, 팔로워, 인기도)
- ✅ 아티스트 인기곡
- ❌ ~~Audio Features (음원 특성)~~ - **더 이상 사용 불가**
- ❌ ~~Audio Analysis (상세 분석)~~ - **더 이상 사용 불가**

#### ReccoBeats API (대체 솔루션)
- ✅ 음원 특성 분석 (Energy, Danceability, Valence 등)
- ✅ 템포, 키, 모드 등 음악 이론적 정보
- ✅ Spotify 트랙 ID를 통한 연동

### 9.4 토큰 관리
- Spotify Access Token은 1시간 후 만료
- 자동으로 새 토큰 발급 로직이 구현되어 있음
- 토큰은 메모리에만 저장 (보안상 안전)

### 9.5 음원 특성(Audio Features) 설명

ReccoBeats API를 통해 얻을 수 있는 음원 특성들:

- **Energy (0.0 ~ 1.0)**: 곡의 강렬함과 활동성 수준
- **Danceability (0.0 ~ 1.0)**: 춤추기 적합한 정도
- **Valence (0.0 ~ 1.0)**: 곡의 긍정적인 감정 정도 (1에 가까울수록 행복하고 밝은 느낌)
- **Acousticness (0.0 ~ 1.0)**: 어쿠스틱 악기 사용 정도
- **Instrumentalness (0.0 ~ 1.0)**: 보컬이 없는 인스트루멘털 정도
- **Speechiness (0.0 ~ 1.0)**: 말하는 듯한 보컬의 비중
- **Liveness (0.0 ~ 1.0)**: 라이브 공연 느낌의 정도
- **Tempo**: BPM (Beats Per Minute)
- **Loudness**: 평균 음량 (dB)
- **Key**: 음조 (0 = C, 1 = C#/Db, 2 = D, ...)
- **Mode**: 장조(1) 또는 단조(0)
- **Time Signature**: 박자 (4/4, 3/4 등)